In [ ]:
!pip install datasets==1.18.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("cfilt/iitb-english-hindi")

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/500k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/cfilt--iitb-english-hindi-911387c6837f8b91/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
source_train_file = open("source_train.txt", "w+", encoding='utf8')
target_train_file = open("target_train.txt", "w+", encoding='utf8')
for translation_pair in dataset["train"]["translation"]:
  source_sentence = translation_pair["en"]
  target_sentence = translation_pair["hi"]
  source_train_file.write(source_sentence.strip("\n") + "\n")
  target_train_file.write(target_sentence.strip("\n") + "\n")
source_train_file.close()
target_train_file.close()

source_valid_file = open("source_valid.txt", "w+", encoding='utf8')
target_valid_file = open("target_valid.txt", "w+", encoding='utf8')
for translation_pair in dataset["validation"]["translation"]:
  source_sentence = translation_pair["en"]
  target_sentence = translation_pair["hi"]
  source_valid_file.write(source_sentence.strip("\n") + "\n")
  target_valid_file.write(target_sentence.strip("\n") + "\n")
source_valid_file.close()
target_valid_file.close()

source_test_file = open("source_test.txt", "w+", encoding='utf8')
target_test_file = open("target_test.txt", "w+", encoding='utf8')
for translation_pair in dataset["test"]["translation"]:
  source_sentence = translation_pair["en"]
  target_sentence = translation_pair["hi"]
  source_test_file.write(source_sentence.strip("\n") + "\n")
  target_test_file.write(target_sentence.strip("\n") + "\n")
source_test_file.close()
target_test_file.close()

In [ ]:
! wc -l source_train.txt target_train.txt source_valid.txt target_valid.txt source_test.txt target_test.txt

  1659083 source_train.txt
  1659083 target_train.txt
      520 source_valid.txt
      520 target_valid.txt
     2507 source_test.txt
     2507 target_test.txt
  3324220 total


In [ ]:
! pip install subword-nmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ['NUM_OF_MERGE_OPERATIONS'] = "16000"

In [ ]:
! cat source_train.txt source_test.txt source_valid.txt > source_full.txt
! cat target_train.txt target_test.txt target_valid.txt > target_full.txt

In [ ]:
! subword-nmt learn-bpe -s $NUM_OF_MERGE_OPERATIONS < source_full.txt > source-bpe.codes
! subword-nmt learn-bpe -s $NUM_OF_MERGE_OPERATIONS < target_full.txt > target-bpe.codes

100% 16000/16000 [02:25<00:00, 110.15it/s]
100% 16000/16000 [03:20<00:00, 79.83it/s] 


In [ ]:

! subword-nmt apply-bpe -c source-bpe.codes < source_train.txt > source_train_bpe.txt
! subword-nmt apply-bpe -c source-bpe.codes < source_valid.txt > source_valid_bpe.txt
! subword-nmt apply-bpe -c source-bpe.codes < source_test.txt > source_test_bpe.txt

! subword-nmt apply-bpe -c target-bpe.codes < target_train.txt > target_train_bpe.txt
! subword-nmt apply-bpe -c target-bpe.codes < target_valid.txt > target_valid_bpe.txt
! subword-nmt apply-bpe -c target-bpe.codes < target_test.txt > target_test_bpe.txt

In [ ]:

! wc -l source-bpe.codes target-bpe.codes

 16001 source-bpe.codes
 16001 target-bpe.codes
 32002 total


In [ ]:
! wc -l source_train_bpe.txt target_train_bpe.txt source_valid_bpe.txt target_valid_bpe.txt source_test_bpe.txt target_test_bpe.txt

  1659083 source_train_bpe.txt
  1659083 target_train_bpe.txt
      520 source_valid_bpe.txt
      520 target_valid_bpe.txt
     2507 source_test_bpe.txt
     2507 target_test_bpe.txt
  3324220 total
